In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

os.environ['THEANO_FLAGS'] = "device=cuda1"

In [ ]:
import shelve
import pprint
import numpy as np
import lasagne
import theano
import theano.tensor as T
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.cm
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from lproc import subset, rmap
import seqtools
from datasets import ch14dataset as dataset
from sltools.utils import gloss2seq, seq2gloss
from sltools.nn_utils import onehot, jaccard, compute_scores
from sltools.postproc import optimize_boundaries, filter_longshort
from IPython.display import set_matplotlib_formats

np.set_printoptions(precision=3)

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
from experiments.hmmvsrnn_reco.a_data import durations, gloss_seqs, tmpdir, \
   train_subset, val_subset, test_subset, vocabulary
from experiments.hmmvsrnn_reco.utils import reload_best_rnn

In [ ]:
if 'EXPERIMENT_NAME' in os.environ:
    experiment_name = os.environ['EXPERIMENT_NAME']
else:
    experiment_name = "rnn_bgr_tc15"

report = shelve.open(os.path.join(tmpdir, experiment_name))

model = report['meta']['model']
modality = report['meta']['modality']
variant = report['meta']['variant']
date = report['meta']['date']
notes = report['meta']['notes']
experiment_name = report['meta']['experiment_name']
args = report['args']
encoder_kwargs = args['encoder_kwargs']

In [ ]:
if modality == "skel":
    from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs
    feat_seqs = [skel_feat_seqs]
elif modality == "bgr":
    from experiments.hmmvsrnn_reco.b_preprocess import bgr_feat_seqs
    feat_seqs = [bgr_feat_seqs]
elif modality == "fusion":
    from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs
    from experiments.hmmvsrnn_reco.b_preprocess import bgr_feat_seqs
    feat_seqs = [skel_feat_seqs, bgr_feat_seqs]
elif modality == "transfer":
    from experiments.hmmvsrnn_reco.b_preprocess import transfer_feat_seqs
    feat_seqs = transfer_feat_seqs(encoder_kwargs['transfer_from'],
                                   encoder_kwargs['freeze_at'])
else:
    raise ValueError()

feat_seqs_train = [subset(f, train_subset) for f in feat_seqs]
gloss_seqs_train = subset(gloss_seqs, train_subset)
durations_train = subset(durations, train_subset)
targets_train = rmap(lambda g, d: gloss2seq(g, d, 0),
                     gloss_seqs_train, durations_train)
feat_seqs_val = [subset(f, val_subset) for f in feat_seqs]
gloss_seqs_val = subset(gloss_seqs, val_subset)
durations_val = subset(durations, val_subset)
targets_val = rmap(lambda g, d: gloss2seq(g, d, 0),
                    gloss_seqs_val, durations_val)
feat_seqs_test = [subset(f, test_subset) for f in feat_seqs]
gloss_seqs_test = subset(gloss_seqs, test_subset)
durations_test = subset(durations, test_subset)
targets_test = rmap(lambda g, d: gloss2seq(g, d, 0),
                    gloss_seqs_test, durations_test)

# Training report

In [ ]:
all_batch_losses = []
all_epoch_losses = []
for i in sorted([e for e in report.keys() if e.startswith("epoch")]):
    r = report[i]
    all_batch_losses += r['batch_losses']
    all_epoch_losses.append(r['epoch_loss'])

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(np.arange(len(all_epoch_losses)), all_epoch_losses, c='red')
n_batches = len(all_batch_losses) // len(all_epoch_losses)
error = np.array([np.std(all_batch_losses[i:i+n_batches]) 
                  for i in range(0, len(all_batch_losses), n_batches)])
# plt.fill_between(np.arange(len(all_epoch_losses)), 
#                  np.maximum(0.00001, all_epoch_losses - error), 
#                  all_epoch_losses + error)
plt.yscale("log")
# plt.semilogy([10 ** (i - 5) for i in range(5)])

In [ ]:
best_epoch, model_dict, predict_fn = reload_best_rnn(report)

print("best epoch: {}".format(best_epoch))

epoch_report = report[best_epoch]
print(epoch_report['train_scores']['jaccard'])
print(epoch_report['train_scores']['framewise'])
print(epoch_report['val_scores']['jaccard'])
print(epoch_report['train_scores']['framewise'] - epoch_report['val_scores']['framewise'])

# Score

In [ ]:
predictions_train = [np.argmax(p, axis=1) for p in predict_fn(feat_seqs_train)]
ji_train, framewise_train, confusion_train = compute_scores(predictions_train, targets_train, vocabulary)

predictions_val = [np.argmax(p, axis=1) for p in predict_fn(feat_seqs_val)]
ji_val, framewise_val, confusion_val = compute_scores(predictions_val, targets_val, vocabulary)

print("JI: {:.4f} / {:.4f}".format(ji_train, ji_val))
print("Accuracy: {:.4f} / {:.4f}".format(framewise_train, framewise_val))

In [ ]:
cnames = [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo']

cmap = matplotlib.cm.viridis
cmap.set_bad(cmap(0.001))

fig, ax1 = plt.subplots(1, 1, figsize=(6, 4))
im = ax1.matshow(confusion_train / np.sum(confusion_train, axis=1, keepdims=True), 
                 interpolation='none', cmap=cmap,
                 clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax1.set_yticks(np.arange(0, 22, 1))
ax1.set_xticks([])
ax1.set_yticklabels(cnames)

divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

fig, ax2 = plt.subplots(1, 1, figsize=(6, 4))
im = ax2.matshow(confusion_val / np.sum(confusion_val, axis=1, keepdims=True), 
                 interpolation='none', cmap='viridis',
                 clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax2.set_yticks(np.arange(0, 22, 1))
ax2.set_xticks([])
ax2.set_yticklabels(cnames)

divider = make_axes_locatable(ax2)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

# Preview prediction

In [ ]:
def preview_seq(proba, ax=None):
    # 21 distinct colors
    
    cmap = np.array([[113,204,0], [209,73,251], [243,255,52], [223,119,255], 
         [139,255,150], [255,66,189], [1,222,201], [255,77,30], 
         [0,149,225], [137,106,0], [0,43,105], [255,230,180], 
         [111,0,66], [0,113,63], [251,177,255], [56,96,0], 
         [160,218,255], [74,0,6], [255,170,172], [0,62,95], 
         [93,43,0]]) / 255
    
    ax = ax or plt.gca()
    l = np.argmax(proba, axis=1)
    for g, start, stop in seq2gloss(l):
        start = max(0, start - 1)
        stop = min(len(proba), stop + 1)
        if g == 0:
            ax.plot(np.arange(start, stop), proba[start:stop, 0], ls=':', c=cmap[0])
        else:
            ax.plot(np.arange(start, stop), proba[start:stop, g], c=cmap[g])
            ax.fill_between(np.arange(start, stop),
                            0, proba[start:stop, g],
                            facecolor=cmap[g],
                            alpha=0.3)
    ax.set_ylim((0, 1.1))
    ax.set_xlim((0, len(proba)))

In [ ]:
s = 62
s = 62

proba = predict_fn([[fseq[s]] for fseq in feat_seqs_val])[0]
labels = onehot(gloss2seq(gloss_seqs_val[s], durations_val[s], 0), 
                np.arange(0, 21))

fig = plt.figure(figsize=(6, 3), dpi=150)
gs = GridSpec(7, 1)
ax = plt.subplot(gs[:4, 0])
ax.set_xticks(np.arange(0, len(proba), 50), minor=True)
preview_seq(np.exp(proba[:]), ax)
ax.set_title("model predictions")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax = plt.subplot(gs[4:, 0])
preview_seq(labels[:] * 1.0,  ax)
ax.set_title("targets")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

fig.tight_layout()

fig.savefig("/home/granger/exp1_preds_vs_tgt.pdf", bbox_inches='tight')

print(gloss_seqs_val[s][:, 0].tolist())

# Score

In [ ]:
# Optimize duration filter

boundaries = optimize_boundaries(targets_val, predictions_val, vocabulary, (30, 100, 301))
print("Optimal range: ", boundaries)
print("FYI the score without is: {:.4f}".format(ji_val))

In [ ]:
# Validation score

ji_filtered_val, accuracy_filtered_val, confusion_filtered_val = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_val], 
    targets_val, vocabulary)
print("validation score: {:.4f}".format(ji_filtered_val))

In [ ]:
# Test score

predictions_test = [np.argmax(p, axis=1) for p in predict_fn(feat_seqs_test)]

ji_filtered_test, accuracy_filtered_test, confusion_filtered_test = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_test], 
    targets_test, vocabulary)
print("testing score: {:.4f}".format(ji_filtered_test))

In [ ]:
# Recap:

print("Accuracy:   {:.4f} / {:.4f} / ?".format(framewise_train, framewise_val))
print("JI:         {:.4f} / {:.4f} / ?".format(ji_train, ji_val))
print("Acc. filt.:      ? / {:.4f} / {:.4f}".format(accuracy_filtered_val, accuracy_filtered_test))
print("JI filt.:        ? / {:.4f} / {:.4f}".format(ji_filtered_val, ji_filtered_test))

recap = {
    "experiment_name": experiment_name,
    "best_epoch": best_epoch,
    "accuracy": (framewise_train, framewise_val, None),
    "ji": (ji_train, ji_val, None),
    "confusion": (confusion_train, confusion_val, None),
    "accuracy_filtered": (None, accuracy_filtered_val, accuracy_filtered_test),
    "ji_filtered": (None, ji_filtered_val, ji_filtered_test),
    "confusion_filtered": (None, confusion_filtered_val, None),
}
report['analysis'] = recap

# Analyse errors

In [ ]:
# Distribution of errors

scores = np.array([jaccard(onehot(l, vocabulary), onehot(p, vocabulary))
                   for l, p in zip(targets_val, predictions_val)])

plt.figure(figsize=(4, 2), dpi=150)
h, bins = np.histogram(scores, np.linspace(0.0, 1, 41))
h = h / np.sum(h) * 40
plt.bar(bins[:-1], h, width=1/40, 
        align='center', color="lightslategray", edgecolor="darkslategray")
plt.xlabel("Sequence Jaccard Index")
plt.ylabel("density")

plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, linestyle='--', linewidth=.5)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.savefig("/home/granger/exp1_ji_density.pdf", bbox_inches='tight')

In [ ]:
# observe badly classified sequences

worst_seqs = np.argsort(scores)[:15]
print("worst sequences:              {}".format(val_subset[worst_seqs]))
print("worst sequences (validation): {}".format(worst_seqs))
print("worst sequences JI:           {}".format(scores[worst_seqs]))

In [ ]:
# nb of false positives out of sequence vocabulary
print(np.mean([len(set(p_) - set(l_)) 
               for p_, l_ in zip(predictions_val, targets_val)],
              axis=0))
print(np.mean([len(set(p_) - set(l_)) 
               for p_, l_ in zip([filter_longshort(p, boundaries, 0) for p in predictions_val], targets_val)],
              axis=0))

In [ ]:
# confusion types
cum_err = np.sum(confusion_val, axis=1) - np.diag(confusion_val)
print("false pos: {}  false neg: {}, mis-class: {}".format(
    cum_err[0], np.sum(confusion_val[1:, 0]), np.sum(cum_err[1:]) - np.sum(confusion_val[1:, 0])))

In [ ]:
6867.0 / (11639.0 + 6083.0 + 6867.0)

In [ ]:
# correlate error with predicted gloss duration

plt.figure(figsize=(7, 2), dpi=150)

prediction_accuracy = [np.sum(l[start:stop] == g) 
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
none_accuracy = [np.sum(l[start:stop] == 0)
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
gloss_d = [stop - start
           for p in predictions_val 
           for (g, start, stop) in seq2gloss(p)
           if g != 0]
gloss_d = np.fmin(gloss_d, 200)

scores_pred = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
scores_none = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
total_d = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
for vp, vn, d in zip(prediction_accuracy, none_accuracy, gloss_d):
    idx = int(d / 5)
    scores_pred[idx] += vp
    scores_none[idx] += vn
    total_d[idx] += d

plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5) + .75, 
              scores_pred / total_d,
              width=1.5,
              color='darkorange')
plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5) - .75, 
              scores_none / total_d,
              width=1.5,
              color='steelblue')
plt.gca().set_xlim([-2.5, 202.5])
plt.gca().set_xticks(np.arange(0, 201, 25))
plt.gca().set_xticklabels([str(i) for i in range(0, 200, 25)] + ["≥ 200"])

plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, linestyle='--', linewidth=.5)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.legend(["predicted class", "non-gesture class"], loc='upper right', bbox_to_anchor=(0.95, 0.95))
plt.xlabel("subsequence duration (based on model prediction)")
plt.ylabel("accuracy")
plt.tight_layout()

plt.savefig("/home/granger/acc_vs_glossduration.pdf", bbox_inches='tight')

In [ ]:
# observe alignment issues

filtered_predictions_val = [filter_longshort(p, boundaries, 0) for p in predictions_val]
gloss_predictions = [
    [(g, start, stop) for (g, start, stop) in seq2gloss(p) if g != 0]
    for p in filtered_predictions_val]

start_offset = []
stop_offset = []

for i in range(len(val_subset)):
    glosses_tgt = gloss_seqs_val[i]
    glosses_pred = gloss_predictions[i]
    for g, a, b in glosses_tgt:
        for g_, a_, b_ in glosses_pred:
            if g_ != g:
                continue
            if min(b, b_) - max(a, a_) < (b - a) / 2:
                continue
            start_offset.append(a_ - a)
            stop_offset.append(b_ - b)
            break

In [ ]:
fig = plt.figure(figsize=(5, 2), dpi=150)
ax = plt.subplot(111)

h, b = np.histogram(np.clip(start_offset, -8, 8), bins=np.arange(-8.5, 9, 1))
h = h / np.sum(h)
ax.bar(b[:-1] + .5 - .15, h, width=.3, color='steelblue')

h, b = np.histogram(np.clip(stop_offset, -8, 8), bins=np.arange(-8.5, 9, 1))
h = h / np.sum(h)
ax.bar(b[:-1] + .5 + .15, h, width=.3, color='darkorange')

ax.set_xlabel('detection offset with groundtruth')
ax.set_ylabel('density')
ax.legend(['beginning', 'end'])

ax.set_xlim((-8.3, 8.3))
ax.set_xticks(np.arange(-8, 9, 2))
ax.set_xticklabels(["≤-8"] + ["+" + str(i) if i > 0 else str(i) for i in  range(-6, 7, 2)] + ["≥+8"])
ax.set_xticks(np.arange(-8, 9), minor=True)

ax.set_axisbelow(True)
ax.yaxis.grid(True, linestyle='--', linewidth=.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

fig.savefig("/home/granger/exp1_gloss_offset.pdf", bbox_inches='tight')

# Analyse model

## Filters

In [ ]:
from sklearn.manifold import TSNE
from sltools.tconv import TemporalConv

tc_l = None
layers = lasagne.layers.get_all_layers(model_dict['l_linout'])

for l in layers:
    if isinstance(l, TemporalConv):
        tc_l = l
        break

W = np.asarray(tc_l.W.eval())
W = np.reshape(np.transpose(W, (0, 2, 1)), (-1, tc_l.filter_size))

norms = np.linalg.norm(W, axis=1)
biggest = np.argsort(norms)[-5000:]
biggest = biggest[np.random.permutation(5000)[:320]]
tsne = TSNE(n_components=1, n_iter=5000, n_iter_without_progress=100, verbose=True)
tsne_order = np.argsort(tsne.fit_transform(W[biggest])[:, 0])
biggest_sorted = biggest[tsne_order]

In [ ]:
plt.figure(figsize=(5, 6), dpi=150)
clim = np.max(np.abs(W))
for i in range(4):
    plt.subplot(1, 4, i + 1)
    img = W[biggest_sorted[i * 80:(i + 1) * 80]]
    img = np.concatenate([img, np.zeros((80 - img.shape[0], img.shape[1]))], axis=0)
    plt.imshow(img, cmap='RdBu', clim=(-clim, clim))
    plt.gca().set_yticks([])
    plt.xticks([0, 14], ['1', '15'])
    plt.gca().set_xticks([4, 9], minor=True)
    
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().set_aspect('equal')

plt.suptitle('time (within window)', y=-0.02)
plt.tight_layout(pad=0.5)

plt.savefig("/home/granger/exp2_tc_weights.pdf", bbox_inches='tight')

In [ ]:
W = np.asarray(layers[2].W.eval())

limits = np.max(abs(W))
fig = plt.figure(figsize=(3, 3), dpi=150)
for i in range(16):
    ax = fig.add_subplot(4, 4, i + 1)
    ax.imshow(W[i, 0], cmap='inferno')
    ax.axis('off')

fig.tight_layout(pad=-2)